In [1]:
! pip install -Iv tensorflow==2.10.0

Using pip 22.3.1 from /env/lib/python3.8/site-packages/pip (python 3.8)
  Using cached tensorflow-2.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.1 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached grpcio-1.51.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached packaging-23.0-py3-none-any.whl (42 kB)
  Using cached libclang-15.0.6.1-py2.py3-none-manylinux2010_x86_64.whl (21.5 MB)
  Using cached numpy-1.24.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached protobuf-3.19.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached flatbuffers-23.1.4-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached absl_py-1.4.0-py3-no

In [2]:
from keras import backend as K
import tensorflow as tf
import os
NUM_PARALLEL_EXEC_UNITS = 8
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=NUM_PARALLEL_EXEC_UNITS, inter_op_parallelism_threads=2,
                       allow_soft_placement=True, device_count={'CPU': NUM_PARALLEL_EXEC_UNITS})
session = tf.compat.v1.Session(config=config)
K.set_session(session)
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["KMP_BLOCKTIME"] = "30"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"

2023-01-17 06:20:26.451032: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 06:20:26.573464: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-17 06:20:26.573482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-17 06:20:26.602349: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-17 06:20:27.292756: W tensorflow/stream_executor/pla

In [3]:
# import tensorflow as tf
import keras
import json
import os
import sys
import random
import datacube
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing

from tools.GEOC_utils import tile_array, untile_array
from tools.Cates_comp_vis_scripts import open_dict_into_partitions, split_data_ID_list_into_partitions
from tools.K_unet import  get_model

In [16]:
from dea_tools.datahandling import pan_sharpen_brovey, load_ard

In [5]:
dc = datacube.Datacube(app="urban_segmentation")

In [6]:
#define model and compile
good_little_model = get_model((128,128), 2)

good_little_model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy",
                              metrics=['accuracy'])

In [7]:
good_little_model

In [8]:
# checkpoint_path = "models/RGB_model_trained_on_C2_landcover/checkpoints_12_dec/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Loads the weights
# good_little_model.load_weights(checkpoint_path)


In [9]:
def normalise(data):
    """"rescale data to be normalised between -1 and +1""""
    
    rescaled = (data - data.mean()) / data.std()

    return rescaled 


In [10]:
class DataGenerator(keras.utils.Sequence):
    '''Generates data for Keras'''
    
    def __init__(self, list_IDs, batch_size=1, dim=(128,128), n_channels=3, #n_channels = bands
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of sites
        
        list_sites_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(list_sites_temp)

        return X, Y
    

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_sites_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        dc = datacube.Datacube(app="urban_segmentation")
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        Y = np.empty((self.batch_size, *self.dim, 1))
        
                # Generate data
        for i, ID in enumerate(list_sites_temp):

            x = np.load(f'training_data//pancro_LC_labeled/{ID}')
            # x = normalise(x)

            y = np.load(f'training_data//pancro_LC_labeled/labels/{ID}')

            X[i,] = x
            Y[i,] = y 

        return X,Y


In [11]:
def do_pand_sharpen(data):
    # Perform Brovey pan-sharpening and return three numpy.arrays
    red_sharpen, green_sharpen, blue_sharpen = pan_sharpen_brovey(
    band_1=data.nbart_red,
    band_2=data.nbart_green,
    band_3=data.nbart_blue,
    pan_band=data.nbart_panchromatic)
    
    
    # Copy the coordinates and dimensions from the original dataset
    time = data.time
    y = data.y
    x = data.x
    coords = [time, y, x]
    dims = ['time', 'y', 'x']

    # Create new data arrays
    red_data_array = xr.DataArray(red_sharpen, coords=coords, dims=dims)
    green_data_array = xr.DataArray(green_sharpen, coords=coords, dims=dims)
    blue_data_array = xr.DataArray(blue_sharpen, coords=coords, dims=dims)

    sharp_rgb = xr.Dataset({'red': red_data_array, 
                            'green': green_data_array, 
                            'blue': blue_data_array}, 
                           coords={'time': time, 'y': y,'x': x})
    
    return sharp_rgb

def make_panchromatic_median(data):
    rescaled = (data - data.mean()) / data.std()
    median = rescaled.median(dim='time', keep_attrs=None)
    return median

In [13]:
list_of_training_files = os.listdir('training_data/pancro_LC_labeled/labels')

['coral_bay_20152141.npy',
 'coral_bay_20152611.npy',
 'north_adeliade_20152374.npy',
 'hobart_20152605.npy',
 'almota_park_20152172.npy',
 'sydney_CBD_20152329.npy',
 'yara_yara_20152616.npy',
 'newcastle_20152633.npy',
 'bendigo_20152230.npy',
 'hobart_20152341.npy',
 'sydney_CBD_2015234.npy',
 'almota_park_20152169.npy',
 'perth_20152517.npy',
 'coral_bay_20152562.npy',
 'sydney_CBD_20152414.npy',
 'sydney_CBD_20152152.npy',
 'newcastle_2015276.npy',
 'bendigo_20152142.npy',
 'hobart_2015291.npy',
 'perth_20152185.npy',
 'lake_mungo_20152436.npy',
 'yara_yara_2015253.npy',
 'hobart_20152469.npy',
 'perth_20152140.npy',
 'hobart_2015216.npy',
 'coral_bay_20152197.npy',
 'north_adeliade_20152452.npy',
 'bendigo_20152510.npy',
 'coral_bay_201520.npy',
 'coral_bay_20152160.npy',
 'perth_20152659.npy',
 'hobart_20152154.npy',
 'sydney_CBD_20152407.npy',
 'coral_bay_20152389.npy',
 'newcastle_20152416.npy',
 'coral_bay_20152331.npy',
 'almota_park_20152556.npy',
 'newcastle_20152376.npy',

In [18]:
#generate data for testing model 
point_x, point_y = (-33.9328, 151.2169)

# Combine central coordinates with buffer values to create the latitude and longitude range
lat = (point_x - 0.2, point_x + 0.2)
lon = (point_y - 0.2, point_y + 0.2)

# Create the 'query' dictionary object, which contains the coordinates defined above
query = {
        "y": lat,
        "x": lon,
        "time": ("2015","2015"),
        }

    # Load DEA Land Cover data from the datacube
# Set the filters to apply
filters = [("opening", 5), ("dilation", 5)]


ARD_lc = load_ard(dc=dc,
        products=['ga_ls8c_ard_3', 'ga_ls7e_ard_3'],
        resampling = 'bilinear',
        output_crs="EPSG:3577",
        measurements=['nbart_blue','nbart_green','nbart_red','nbart_panchromatic'],
        resolution=(-15, 15),
        group_by='solar_day',
        ls7_slc_off=False,
        mask_filters=filters,
        **query
        )

#     print(Geomedian)
sharp_rgb = do_pand_sharpen(ARD_lc)

sharp_median = make_panchromatic_median(sharp_rgb)




# #change order of dementions to what the GEOC utils expect
sharp_median = sharp_median.transpose('x','y','variable')

og_x = sharp_median.shape[0]
og_y = sharp_median.shape[1]

tile_geomedian = tile_array(sharp_median, xsize=128, ysize=128, overlap=0.5)


Finding datasets
    ga_ls8c_ard_3
    ga_ls7e_ard_3 (ignoring SLC-off observations)
Applying morphological filters to pixel quality mask: [('opening', 5), ('dilation', 5)]
Applying pixel quality/cloud mask (oa_fmask)
Loading 46 time steps


/home/jovyan/Tools/dea_tools/datahandling.py:476: UserWarning: As of `dea_tools` v0.3.0, pixel quality masks are inverted before being passed to `mask_filters` (i.e. so that good quality/clear pixels are False and poor quality pixels/clouds are True). This means that 'dilation' will now expand cloudy pixels, rather than shrink them as in previous versions.
  warnings.warn(
/env/lib/python3.8/site-packages/rasterio/warp.py:346: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


KeyboardInterrupt: 

In [ ]:
#create a call back
new_checkpoint_path = "models/jan_2023/cp_{epoch:04d}.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=new_checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq=500)


In [ ]:
class PerformancePlotCallback(keras.callbacks.Callback):
    def __init__(self, test_gm, model_name):
        self.test_gm = test_gm
        self.model_name = model_name
        
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.test_gm)
        result = xr.DataArray(untile_array(predictions, (og_x, og_y, 2), overlap=0.5, smooth_blending=False).transpose(2,1,0))
    
        plt.figure()
        plt.imshow(result[1])
        plt.title(f'Prediction Visualization Keras Callback - Epoch: {epoch}')
        plt.savefig(f'{image_output_path}model_result_1p_E_{epoch}.png')
        plt.clf()

In [ ]:
#load txt file containing dict of training areas with dates

image_output_path = 'training_process_figs/'

#     short_list_of_training_files = list_of_training_files

partitions = split_data_ID_list_into_partitions(list_of_training_files)

# Parameters
params = {'dim': (128,128),
          'batch_size': 100,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}


training_generator = DataGenerator(partitions['train'], **params)
validation_generator = DataGenerator(partitions['validation'], **params)

# Train model on dataset
model_history = good_little_model.fit_generator(generator=training_generator, epochs=10,
                    validation_data=validation_generator,
                    callbacks=[cp_callback],
                    use_multiprocessing=True,
                    workers=8)



test model

In [ ]:
predictions = good_little_model.predict(tile_geomedian)

result = xr.DataArray(untile_array(predictions, (og_x, og_y, 2), overlap=0.5, smooth_blending=True).transpose(2,1,0))

plt.figure()
plt.imshow(result[0])


In [ ]:
result